# Import

In [1]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os
import visa
rm = visa.ResourceManager()

# Load modules

In [2]:
try: 
    pulsedmaster
except NameError:
    manager.startModule('logic', 'pulsedmaster')
try: 
    scanner
except NameError:
    manager.startModule('logic', 'scanner')
try: 
    optimizer
except NameError:
    manager.startModule('logic', 'optimizer')

# Method definitions

### Utility methods:

In [3]:
def write_to_logfile(nametag, timestamp,name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = save.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

### Measurement methods:

In [55]:
def refocus_position():
    # switch on laser
    pulsedmaster.load_asset_into_channels('Laser_On')
    while pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.toggle_pulse_generator(True)
    # perform refocus
    scanner.stop_scanning()
    crosshair_pos = scanner.get_position()
    optimizer.start_refocus(initial_pos=crosshair_pos)
    while optimizer.getState() == 'idle':
        time.sleep(0.2)
    while optimizer.getState() != 'idle':
        time.sleep(0.2)
    scanner.set_position('optimizer', x=optimizer.optim_pos_x, y=optimizer.optim_pos_y, z=optimizer.optim_pos_z, a=0.0)
    time.sleep(1)
    # switch off laser
    pulsedmaster.toggle_pulse_generator(False)
    
    # write new position to log file return
    timestamp = str(datetime.datetime.now())
    new_x = optimizer.optim_pos_x
    new_y = optimizer.optim_pos_y
    new_z = optimizer.optim_pos_z
#    drift = np.sqrt((new_x-position['x'])**2 + (new_y-position['y'])**2 + (new_z-position['z'])**2)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=new_x, y_pos=new_y, z_pos=new_z)
    return new_x, new_y, new_z

def refocus_poi(poi=None, max_drift=0.3,min_counts=1e5):
    if poi==None:
        return False
    pulsedmaster.load_asset_into_channels('Laser_On')
    while pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.toggle_pulse_generator(True)
    # perform refocus
    scanner.stop_scanning()
    poimanager.go_to_poi(poi)
    pos_start = scanner.get_position()
    setup['NV_name']=poimanager.poi_list[poi]._name
    poimanager.optimise_poi(poi)
    while optimizer.getState() == 'idle':
        time.sleep(0.2)
    while optimizer.getState() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scanner.get_position()
    print(counter.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counter.countdata.mean()<min_counts):
        scanner.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanager.set_new_position(poi,pos_start)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True
    
def do_rabi(generate_new=True, save_tag=None):
    # generate ensemble object
    rabi['tau_res'] = rabi['length'] / (rabi['points'] - 1)
    tau_arr = np.arange(rabi['points']) * rabi['tau_res']
    if generate_new:
        sequencegenerator.delete_ensemble('Rabi')
        sequencegenerator.generate_rabi('Rabi', tau_arr[0], rabi['tau_res'], rabi['points'], rabi['mw_freq'], 
                                        rabi['mw_amp'], 'a_ch1',setup['laser_length'] , 0.0, setup['aom_delay'], 
                                        setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'Rabi' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of Rabi in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('Rabi', True)
    else:
        pulsedmaster.load_asset_into_channels('Rabi')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(tau_arr, rabi['points'], 100e-6, [], False)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'] )
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement()
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < rabi['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.manually_pull_data() 
    time.sleep(1)
    pulsedmaster.stop_measurement()
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    if save_tag is None:
        pulsedmaster.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)
    else:
        pulsedmaster.save_measurement_data('s',save_tag,True)
    time.sleep(2)
    # do fit
    x, y, param_dict = pulsedmeasurement.do_fit('Sine')
    new_freq = param_dict.params['frequency'].value
    contrast = 2.*param_dict.params['amplitude'].value
    period = 1./new_freq
    
    # write logfile
    write_to_logfile('Rabi_log', str(datetime.datetime.now()),name= setup['NV_name'],amplitude=rabi['mw_amp'], period=period,
                     frequency=new_freq, contrast=contrast)
    return user_terminated, period, contrast, new_freq

def do_odmr(generate_new=True,save_data=False):
    # generate ensemble object
    start_freq = odmr['resonance_freq']-(odmr['points']//2)*odmr['freq_res']
    freq_arr = start_freq + np.arange(odmr['points']) * odmr['freq_res']
    if generate_new:
        sequencegenerator.delete_ensemble('PulsedODMR')
        sequencegenerator.generate_pulsedodmr('PulsedODMR', odmr['pi_length'], start_freq, odmr['freq_res'], 
                                              odmr['points'], odmr['mw_amp'], 'a_ch1', setup['laser_length'] , 
                                              0.0, setup['aom_delay'], setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'PulsedODMR' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of PulsedODMR from notebook did not work.')
            return
        pulsedmaster.sample_block_ensemble('PulsedODMR', True)
    else:
        pulsedmaster.load_asset_into_channels('PulsedODMR')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    # delete old fit
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(freq_arr, odmr['points'], 100e-6, [], False)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay']+setup['laser_length'] )
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement()
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < odmr['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(1)
    pulsedmaster.manually_pull_data() 
    time.sleep(1)
    pulsedmaster.stop_measurement()
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
#    pulsedmaster.save_measurement_data('Hz','PulsedODMR'+setup['NV_name'])
    time.sleep(2)
    if odmr['fit_function']=='Lorentian':
        x, y, param_dict= pulsedmeasurement.do_fit('Lorentian')
        new_freq = param_dict.params['center'].value
        contrast = param_dict.params['contrast'].value
        linewidth = param_dict.params['fwhm'].value
    if odmr['fit_function']=='15N':
        x, y, param_dict = pulsedmeasurement.do_fit('15N')
        maximal_contrast = param_dict.params['l0_amplitude'].value
        new_freq = param_dict.params['l0_center'].value
        contrast = maximal_contrast
        linewidth = param_dict.params['l0_fwhm'].value
        
    freq_shift = new_freq - odmr['resonance_freq']
    if save_data:
        pulsedmaster.save_measurement_data('Hz','ODMR_'+setup['NV_name'],True)
    # write logfile
    write_to_logfile('PulsedODMR_log', str(datetime.datetime.now()),name= setup['NV_name'], frequency=new_freq,
                     contrast=contrast,linewidth=linewidth, shift=freq_shift)
    return user_terminated, new_freq, contrast, freq_shift, linewidth

def do_hahnecho(generate_new=True, save_tag=None):
    # generate ensemble object
    hahnecho['tau_res'] = (hahnecho['length']-hahnecho['start_tau']) / (hahnecho['points'])
    tau_arr = hahnecho['start_tau']+np.arange(hahnecho['points']) * hahnecho['tau_res']
    if generate_new:
        sequencegenerator.delete_ensemble('HahnEcho')
        sequencegenerator.generate_hahnecho('HahnEcho', hahnecho['rabi_period'], hahnecho['mw_freq'],hahnecho['mw_amp'], 
                                             tau_arr[0],hahnecho['tau_res'],hahnecho['points'], 
                                             'a_ch1', setup['laser_length'] , 0.0, setup['aom_delay'], 
                                             setup['wait_length'], setup['seq_trig'], setup['gate'],hahnecho['alternating'])
        if 'HahnEcho' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of HahnEcho in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('HahnEcho', True)
    else:
        pulsedmaster.load_asset_into_channels('HahnEcho')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    if hahnecho['alternating']:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, 2*hahnecho['points'], 100e-6, [], hahnecho['alternating'])
    else:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, hahnecho['points'], 100e-6, [], hahnecho['alternating'])
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement()
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hahnecho['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.manually_pull_data() 
    time.sleep(1)
    pulsedmaster.stop_measurement()
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    if save_tag is None:
        pulsedmaster.save_measurement_data('s','Hahnecho_'+setup['NV_name'],True)
    else:
        pulsedmaster.save_measurement_data('s',save_tag,True)
    time.sleep(2)
    # do fit
    x, y, param_dict = pulsedmeasurement.do_fit('T2')
    T2 = param_dict.params['lifetime'].value
    contrast = param_dict.params['amplitude'].value
    
    # write logfile
    write_to_logfile('Hahnecho_log', str(datetime.datetime.now()),name= setup['NV_name'],T2=T2, contrast=contrast)
    return user_terminated,T2, contrast

def do_HHamp(generate_new=True, load_tag='' ,save_tag=''):
    # generate ensemble object
    amp_interval = (hhamp['points'] - 1) * hhamp['amp_res']
    amp_start = hhamp['resonance_amp'] - amp_interval / 2
    amp_arr = amp_start + np.arange(hhamp['points']) * hhamp['amp_res']
    if generate_new:
        sequencegenerator.delete_ensemble('HHamp')
        sequencegenerator.generate_HHamp('HHamp', hhamp['rabi_period'], hhamp['sl_length'], hhamp['mw_freq'],
                                         hhamp['pulse_amp'], amp_start, hhamp['amp_res'], hhamp['points'], 
                                         'a_ch1', setup['laser_length'], 0.0, setup['aom_delay'],
                                         setup['wait_length'], setup['seq_trig'], setup['gate'])
        # sample, upload and load waveform
        if 'HHamp' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of HHamp in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('HHamp', True)
    else:
        pulsedmaster.load_asset_into_channels('HHamp')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(amp_arr, 2 * hhamp['points'], 100e-6, [], True)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(load_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hhamp['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    time.sleep(2)
    # do fit
    #x, y, param_dict= pulsedmeasurement.do_fit('Lorentian(pos)')
    #new_amp = param_dict.params['center'].value
    #contrast = param_dict.params['contrast'].value
    #linewidth = param_dict.params['fwhm'].value
    #amp_shift = new_amp - hhamp['resonance_amp']    
    return user_terminated

def do_HHtau(generate_new=True, load_tag='' ,save_tag=''):
    # generate ensemble object
    tau_array = hhtau['tau_start']+np.arange(hhtau['points']) * hhtau['tau_res']
    if generate_new:
        sequencegenerator.delete_ensemble('HHtau')
        sequencegenerator.generate_HHtau('HHtau', hhtau['rabi_period'], hhtau['sl_amp'], hhtau['mw_freq'],
                                         hhtau['pulse_amp'], hhtau['tau_start'], hhtau['tau_res'], hhtau['points'], 
                                         'a_ch1', setup['laser_length'], 0.0, setup['aom_delay'],
                                         setup['wait_length'], setup['seq_trig'], setup['gate'])
        # sample, upload and load waveform
        if 'HHtau' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of HHtau in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('HHtau', True)
    else:
        pulsedmaster.load_asset_into_channels('HHtau')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(tau_array, 2 * hhtau['points'], 100e-6, [], True)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(load_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hhtau['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    time.sleep(2)
    return user_terminated


def do_xy8(generate_new=True, save_tag='',load_tag=''):
    # generate ensemble object
    tau_arr = xy8['tau_start'] + np.arange(xy8['points']) * xy8['tau_incr']
    if generate_new:
        sequencegenerator.delete_ensemble('XY8')
        sequencegenerator.generate_xy8_tau('XY8', xy8['rabi_period'], xy8['mw_freq'], xy8['mw_amp'],
                                            xy8['tau_start'], xy8['tau_incr'], xy8['points'], 
                                            xy8['order'], 'a_ch1', setup['laser_length'], 0.0, 
                                            setup['aom_delay'], setup['wait_length'], setup['seq_trig'], 
                                            setup['gate'],xy8['alternating'])
        # sample, upload and load waveform
        if 'XY8' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of XY8 in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('XY8', True)
        
    else:
        pulsedmaster.load_asset_into_channels('XY8')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    if xy8['alternating']:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, 2*xy8['points'], 100e-6, [],xy8['alternating'])
    else:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, xy8['points'], 100e-6, [],xy8['alternating'])
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    pulsedmeasurement.measurement_tag = save_tag
    # perform measurement
    pulsedmaster.start_measurement(load_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < xy8['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmaster.save_measurement_data('s','XY8-'+str(xy8['order'])+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_xy8order(generate_new=True, save_tag='',load_tag=''):
    # generate ensemble object
    order_arr = xy8order['order_start'] + np.arange(xy8order['points']) * xy8order['order_incr']
    if generate_new:
        sequencegenerator.delete_ensemble('XY8')
        sequencegenerator.generate_xy8_order('XY8', xy8order['rabi_period'], xy8order['mw_freq'], xy8order['mw_amp'],
                                            xy8order['tau'], xy8order['order_start'], xy8order['order_incr'], 
                                            xy8order['points'], 'a_ch1', setup['laser_length'], 0.0, 
                                            setup['aom_delay'], setup['wait_length'], setup['seq_trig'], 
                                            setup['gate'],True)
        # sample, upload and load waveform
        if 'XY8' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of XY8 in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('XY8', True)
        
    else:
        pulsedmaster.load_asset_into_channels('XY8')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(order_arr*xy8order['tau']*8, 2*xy8order['points'], 100e-6, [],True)
      
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(load_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < xy8order['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmaster.save_measurement_data('s','XY8-T2_decay_'+str(round(xy8order['tau']*1e9,1))
                                           +'ns_tau'+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_pptau(generate_new=True, save_tag='',load_tag=''):
    # generate ensemble object
    tau_arr = pptau['tau_start'] + np.arange(pptau['points']) * pptau['tau_incr']
    if generate_new:
        sequencegenerator.delete_ensemble('PulsePol')
        sequencegenerator.generate_Pol20('PulsePol', pptau['rabi_period'], pptau['mw_freq'], pptau['mw_amp'],
                                            pptau['tau_start'], pptau['tau_incr'], pptau['points'], 
                                            pptau['order'], 'a_ch1', setup['laser_length'], 0.0, 
                                            setup['aom_delay'], setup['wait_length'], setup['seq_trig'], 
                                            setup['gate'],pptau['alternating'])
        # sample, upload and load waveform
        if 'PulsePol' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of PulsePol in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('PulsePol', True)
        
    else:
        pulsedmaster.load_asset_into_channels('PulsePol')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    if pptau['alternating']:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, 2*pptau['points'], 100e-6, [],pptau['alternating'])
    else:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, pptau['points'], 100e-6, [],pptau['alternating'])
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(load_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < pptau['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmaster.save_measurement_data('s','PulsePol-'+str(pptau['order'])+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_pporder(generate_new=True, save_tag='',load_tag=''):
    # generate ensemble object
    order_arr = pporder['order_start'] + np.arange(pporder['points']) * pporder['order_incr']
    if generate_new:
        sequencegenerator.delete_ensemble('PulsePol')
        sequencegenerator.generate_Pol20_order('PulsePol', pporder['rabi_period'], pporder['mw_freq'], pporder['mw_amp'],
                                            pporder['tau'], pporder['order_start'], pporder['order_incr'], 
                                            pporder['points'], 'a_ch1', setup['laser_length'], 0.0, 
                                            setup['aom_delay'], setup['wait_length'], setup['seq_trig'], 
                                            setup['gate'])
        # sample, upload and load waveform
        if 'PulsePol' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of PulsePol in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('PulsePol', True)
        
    else:
        pulsedmaster.load_asset_into_channels('PulsePol')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(order_arr*pporder['tau']*2, 2*pporder['points'], 100e-6, [],True)
      
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + setup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(load_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < pporder['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmaster.save_measurement_data('s','PulsePol-T2_decay_'+str(round(pporder['tau']*1e9,1))
                                           +'ns_tau'+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_multi_measurement(poi_list,measurement_list):
    for poi in poi_list:
        if poi != 'crosshair' and poi !='sample':
            setup['NV_name']=poimanager.poi_list[poi]._name
            #checking stop condition
            user_terminated = False
            #Refocus current POI and calculate sample shift(all POI are shifted)    
            if 'odmr' in measurement_list:
                for i in range(3):
                    if refocus_poi(poi):
                        result=do_odmr()
                        user_terminated=result[0]
                        rabi['mw_freq']=xy8['mw_freq']=xy8order['mw_freq']=pptau['mw_freq']=pporder['mw_freq']=hahnecho['mw_freq']=result[1]
                        break
            if user_terminated:
                return True
            if 'rabi' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        result=do_rabi()
                        user_terminated=result[0]
                        hahnecho['mw_amp']=rabi['mw_amp']
                        xy8['mw_amp']=xy8order['mw_amp']=pporder['mw_amp']=pptau['mw_amp']=rabi['mw_amp']
                        xy8['rabi_period']=xy8order['rabi_period']=pporder['rabi_period']=pptau['rabi_period']=result[1]  
                        hahnecho['rabi_period']=result[1] 
                        break
            if user_terminated:
                return True
            if 'hahnecho' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        result=do_hahnecho()
                        user_terminated=result[0]
                        break
            
            if user_terminated:
                return True
            if 'xy8' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        if do_xy8_refocus(poi):
                            return True
                        break
            if user_terminated:
                return True
            if 'hhamp' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        if do_HHamp_refocus(poi):
                            return True
                        break 
            if user_terminated:
                return True
            if 'hhtau' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        if do_HHtau_refocus(poi):
                            return True
                        break
            if user_terminated:
                return True
            if 'pptau' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        if do_pptau_refocus(poi):
                            return True
                        break 
            if user_terminated:
                return True
            if 'pporder' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        if do_pporder_refocus(poi):
                            return True
                        break 
            if user_terminated:
                return True
            if 'xy8order' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi):
                        if do_xy8order_refocus(poi):
                            return True
                        break 
    return False
                                       
                

### refocus methods:

In [5]:
def do_xy8_refocus(poi):
    end_measure=False
    refocus_poi(poi)
    xy8_total_time = xy8['measurement_time']
    xy8_runtime = 0.0
    xy8['measurement_time'] = xy8['refocus_interval']
    end_measure = do_xy8(True, 'xy8_refocus','')
    xy8_runtime += xy8['refocus_interval']
    while xy8_total_time > xy8_runtime:
        for i in range(3):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_xy8(False,'xy8_refocus', 'xy8_refocus')
        xy8_runtime += xy8['refocus_interval']
        if end_measure:
            break
    xy8['measurement_time'] = xy8_total_time
    # save measurement
    pulsedmaster.save_measurement_data('s','XY8-'+str(xy8['order'])+'_'+setup['NV_name'],True)
    write_to_logfile('xy8_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=xy8_runtime)
    time.sleep(2)
    return end_measure

def do_xy8order_refocus(poi):
    end_measure=False
    refocus_poi(poi)
    xy8order_total_time = xy8order['measurement_time']
    xy8order_runtime = 0.0
    xy8order['measurement_time'] = xy8order['refocus_interval']
    end_measure = do_xy8order(True, 'xy8order_refocus','')
    xy8order_runtime += xy8order['refocus_interval']
    while xy8order_total_time > xy8order_runtime:
        for i in range(3):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_xy8order(False,'xy8order_refocus', 'xy8order_refocus')
        xy8order_runtime += xy8order['refocus_interval']
        if end_measure:
            break
    xy8order['measurement_time'] = xy8order_total_time
    # save measurement
    pulsedmaster.save_measurement_data('s','XY8-T2_decay_'+str(round(xy8order['tau']*1e9,1))+
                                       'ns_tau'+'_'+setup['NV_name'],True)
    write_to_logfile('xy8order_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=xy8order_runtime)
    time.sleep(2)
    return end_measure

def do_pptau_refocus(poi):
    end_measure=False
    refocus_poi(poi)
    pptau_total_time = pptau['measurement_time']
    pptau_runtime = 0.0
    pptau['measurement_time'] = pptau['refocus_interval']
    end_measure = do_pptau(True, 'pptau_refocus','')
    pptau_runtime += pptau['refocus_interval']
    while pptau_total_time > pptau_runtime:
        for i in range(3):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_pptau(False,'pptau_refocus', 'pptau_refocus')
        pptau_runtime += pptau['refocus_interval']
        if end_measure:
            break
    pptau['measurement_time'] = pptau_total_time
    # save measurement
    pulsedmaster.save_measurement_data('s','PulsePol-'+str(pptau['order'])+'_'+setup['NV_name']+'_amp_'
                                       +str(pptau['mw_amp'])+'V',True)
    write_to_logfile('pptau_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=pptau_runtime)
    time.sleep(2)
    return end_measure

def do_pporder_refocus(poi):
    end_measure=False
    refocus_poi(poi)
    pporder_total_time = pporder['measurement_time']
    pporder_runtime = 0.0
    pporder['measurement_time'] = pporder['refocus_interval']
    end_measure = do_pporder(True, 'pporder_refocus','')
    pporder_runtime += pporder['refocus_interval']
    while pporder_total_time > pporder_runtime:
        for i in range(3):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_pporder(False,'pporder_refocus', 'pporder_refocus')
        pporder_runtime += pporder['refocus_interval']
        if end_measure:
            break
    pporder['measurement_time'] = pporder_total_time
    # save measurement
    pulsedmaster.save_measurement_data('s','PulsePol-T2_decay_'+str(round(pporder['tau']*1e9,1))+
                                       'ns_tau'+'_'+setup['NV_name'],True)
    write_to_logfile('pporder_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=pporder_runtime)
    time.sleep(2)
    return end_measure

def do_HHamp_refocus(poi):
    refocus_poi(poi)
    end_measure, odmr_freq, odmr_contrast, odmr_shift, odmr_linewidth = do_odmr(True,False)
    if end_measure:
        return
    hhamp['mw_freq'] = odmr_freq
    odmr['resonance_freq']=odmr_freq
    hhamp_total_time = hhamp['measurement_time']
    hhamp_runtime = 0.0
    hhamp['measurement_time'] = hhamp['refocus_interval']
    end_measure = do_HHamp(True, '','HHamp_refocus')
    hhamp_runtime += hhamp['refocus_interval']
    while hhamp_total_time > hhamp_runtime:
        for i in range(3):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure, odmr_freq, odmr_contrast, odmr_shift, odmr_linewidth = do_odmr(True,False)
        if end_measure:
            break
        hhamp['mw_freq'] = odmr_freq
        odmr['resonance_freq']=odmr_freq
        end_measure = do_HHamp(True,'HHamp_refocus' ,'HHamp_refocus')
        hhamp_runtime += hhamp['refocus_interval']
        if end_measure:
            break
    hhamp['measurement_time'] = hhamp_total_time
    
    # do fit
    x, y, param_dict= pulsedmeasurement.do_fit('Lorentian(pos)')
    new_amp = param_dict.params['center'].value
    contrast = param_dict.params['contrast'].value
    linewidth = param_dict.params['fwhm'].value 
    # save measurement
    pulsedmaster.save_measurement_data('V','HHamp_withrefocus_final'
                                       +setup['NV_name']+'_'+str(round(hhamp['sl_length']/1e-6)) +'mus_SL',True)
    
    write_to_logfile('HHamp_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=hhamp_runtime, 
                     resonant_amp=new_amp,linewidth=linewidth)
    time.sleep(2)
    return end_measure

def do_HHtau_refocus(poi):
    refocus_poi(poi)
    end_measure, odmr_freq, odmr_contrast, odmr_shift, odmr_linewidth = do_odmr(True, False)
    if end_measure:
        return
    hhtau['mw_freq'] = odmr_freq
    odmr['resonance_freq']=odmr_freq
    hhtau_total_time = hhtau['measurement_time']
    hhtau_runtime = 0.0
    hhtau['measurement_time'] = hhtau['refocus_interval']
    end_measure = do_HHtau(True, '','HHtau_refocus')
    hhtau_runtime += hhtau['refocus_interval']
    while hhtau_total_time > hhtau_runtime:
        for i in range(3):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        end_measure, odmr_freq, odmr_contrast, odmr_shift, odmr_linewidth = do_odmr(True)
        if end_measure:
            break
        
        hhtau['mw_freq'] = odmr_freq
        odmr['resonance_freq']=odmr_freq
        end_measure = do_HHtau(True,'HHtau_refocus' ,'HHtau_refocus')
        hhtau_runtime += hhtau['refocus_interval']
        if end_measure:
            break
    hhtau['measurement_time'] = hhtau_total_time
    # save measurement
    pulsedmaster.save_measurement_data('s','HHtau_withrefocus_final'
                                       +setup['NV_name']+'_'+str(round(hhtau['sl_amp']/1e-6)) +'amp_SL',True)
    
    write_to_logfile('HHtau_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=hhtau_runtime)
    time.sleep(2)
    return end_measure



# Input parameters

In [72]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurement.sample_rate
setup['fc_binwidth'] = pulsedmeasurement.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']='d_ch3'
setup['gate']='d_ch2'
setup['NV_name']='NV14'
setup['laser_length'] = 3.0e-6

# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 1.0
amp_dict['a_ch2'] = 1.0
sequencegenerator.amplitude_dict = amp_dict
pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

# measurement start input parameters

# for Rabi:
rabi = OrderedDict()
rabi['mw_amp'] = 0.5
rabi['mw_freq'] = 1040e6
rabi['length'] = 140e-9
rabi['points'] = 50
rabi['measurement_time'] = 60
rabi['refocus_interval'] = 900

# for HHtau
hhtau = OrderedDict()
hhtau['pulse_amp'] = 0.5
hhtau['mw_freq'] = 1020e6
hhtau['rabi_period'] = 187e-9
hhtau['sl_amp'] = 0.259
hhtau['tau_start']=1e-6
hhtau['tau_res'] = 5e-6
hhtau['points'] = 60
hhtau['measurement_time'] = 10800
hhtau['refocus_interval'] = 300

# for HHamp:
hhamp = OrderedDict()
hhamp['pulse_amp'] = 0.5
hhamp['mw_freq'] = 1012.5e6
hhamp['rabi_period'] = 282e-9
hhamp['sl_length'] = 10e-6
hhamp['resonance_amp'] = 0.38
hhamp['amp_res'] = 0.001
hhamp['points'] = 40
hhamp['measurement_time'] = 3600
hhamp['refocus_interval'] = 300

# for single frequency XY8-N:
xy8 = OrderedDict()
xy8['mw_amp'] = 0.5
xy8['mw_freq'] = 1020e6
xy8['rabi_period'] = 22e-9
xy8['order'] = 24
xy8['tau_start'] = 175e-9
xy8['tau_incr'] = 0.2e-9
xy8['points'] = 30
xy8['measurement_time'] = 7200
xy8['refocus_interval'] = 240
xy8['alternating']=True

# for PulsePol spectrum (pptau):
pptau = OrderedDict()
pptau['mw_amp'] = 0.5
pptau['mw_freq'] = 1023e6
pptau['rabi_period'] = 15e-9
pptau['order'] = 20 # the N of 2N --> whole sequnece has a length of 2*tau*order
pptau['tau_start'] = 550e-9
pptau['tau_incr'] = 1e-9
pptau['points'] = 40
pptau['measurement_time'] = 7200
pptau['refocus_interval'] = 240
pptau['alternating']=True

# for PulsePol decay (pporder):
pporder = OrderedDict()
pporder['mw_amp'] = 0.5
pporder['mw_freq'] = 1020e6
pporder['rabi_period'] = 15e-9
pporder['tau'] = 530e-9 
pporder['order_start'] = 1
pporder['order_incr'] = 1
pporder['points'] = 20
pporder['measurement_time'] = 3600
pporder['refocus_interval'] = 240

# for XY8 decay (xy8order):
xy8order = OrderedDict()
xy8order['mw_amp'] = 0.5
xy8order['mw_freq'] = 1014e6
xy8order['rabi_period'] = 15e-9
xy8order['tau'] = 200e-9 
xy8order['order_start'] = 1
xy8order['order_incr'] = 1
xy8order['points'] = 20
xy8order['measurement_time'] = 3600
xy8order['refocus_interval'] = 240

# for pulsed ODMR:
odmr = OrderedDict()
odmr['mw_amp'] = 0.05
odmr['pi_length'] = 2454e-9/2.
odmr['resonance_freq'] = 1014e6
odmr['freq_res'] = 0.2e6
odmr['points'] = 60
odmr['measurement_time'] = 60
odmr['fit_function']='15N'
#odmr['fit_function']='Lorentian'

hahnecho=OrderedDict()
hahnecho['mw_amp']=0.5
hahnecho['mw_freq']=1040e6
hahnecho['rabi_period']=13.7e-9
hahnecho['points']=30
hahnecho['length']=20e-6
hahnecho['start_tau']=50e-9
hahnecho['alternating']=False
hahnecho['measurement_time']=300

sameforall=False
general=OrderedDict()
general['mw_amp']=0.25
general['mw_freq']=1020e6
if sameforall:
    hahnecho['mw_amp']=xy8['mw_amp']=rabi['mw_amp']=general['mw_amp']
    hahnecho['mw_freq']=xy8['mw_freq']=rabi['mw_freq']=general['mw_freq']

In [51]:
amp_array=[0.5,0.37005,0.4,0.36,0.35,0.37]
for amp in amp_array:
    rabi['mw_amp']=amp
    do_multi_measurement(['poi_20170127_1335_14_251083'],['rabi'])


140887.333333 9.131878836343669e-08
Rabi
seq_trigger
143569.333333 3.713562602429321e-08
Rabi
seq_trigger
133365.166667 1.2544336278514794e-08
Rabi
seq_trigger
143064.333333 8.09081010152144e-09
Rabi
seq_trigger
142411.333333 1.583991485466267e-08
Rabi
seq_trigger
144325.0 1.3525595278964105e-09
Rabi
seq_trigger


In [58]:
rabi['mw_amp']=0.5
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','hhamp'])

145757.666667 3.198548833860878e-08
PulsedODMR
seq_trigger
149363.666667 1.0873316484902171e-08
Rabi
seq_trigger
151232.5 2.0583409480911976e-09
149933.333333 6.409759470894403e-09
PulsedODMR
seq_trigger
HHamp
seq_trigger
156761.833333 5.6779375742001145e-08
PulsedODMR
seq_trigger
HHamp
seq_trigger
147412.0 1.5257016419640895e-08
PulsedODMR
seq_trigger
HHamp
seq_trigger


True

In [74]:
rabi['mw_freq'] =xy8order['mw_freq']= 1008e6
do_multi_measurement(['poi_20170127_1335_14_251083'],['rabi','xy8order'])

85472.3333333 2.1900143642101228e-07
183742.166667 9.232196965228262e-08
Rabi
seq_trigger
162649.0 4.1580763567775867e-08
170820.5 1.418171750409839e-09
XY8
seq_trigger
173609.666667 9.007579757984612e-08
179878.5 1.556086797858726e-07
182285.0 6.134353994173485e-08
82127.3333333 1.6965789372629081e-07
64572.6666667 1.408433665732418e-09
117683.333333 8.663054845070497e-08


True

In [ ]:
#NV21
xy8order['tau'] = 178e-9 # the N of 2N --> whole sequnece has a length of 2*tau*order
xy8order['order_start'] = 1
xy8order['order_incr'] = 1
xy8order['points'] = 35
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','xy8order'])
xy8order['tau'] = 165e-9
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','xy8order'])
xy8order['tau'] = 200e-9
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','xy8order'])
#NV14
xy8order['order_incr'] = 2
xy8order['points'] = 30
xy8order['tau'] = 178e-9
do_multi_measurement(['poi_20170127_1402_38_232389'],['odmr','rabi','xy8order'])
xy8order['tau'] = 165e-9
do_multi_measurement(['poi_20170127_1402_38_232389'],['odmr','rabi','xy8order'])
xy8order['tau'] = 200e-9
do_multi_measurement(['poi_20170127_1402_38_232389'],['odmr','rabi','xy8order'])

In [71]:
all_NV=['poi_20170127_1346_49_682718','poi_20170127_1402_38_232389','poi_20170127_1335_14_251083']
#NV105, NV14, NV21
#rabi['mw_freq'] =hahnecho['mw_freq']= 1014e6
#do_multi_measurement(['poi_20170127_1402_38_232389'],['rabi','hahnecho'])
rabi['mw_freq'] =hahnecho['mw_freq']= 1008.6e6
hahnecho['length']=10e-6
do_multi_measurement(['poi_20170127_1335_14_251083'],['rabi','hahnecho'])

188297.166667 8.085205780750487e-08
Rabi
seq_trigger
166026.333333 2.16108789700846e-08
HahnEcho
seq_trigger


False

In [ ]:
#NV21
pporder['tau'] = 572e-9 # the N of 2N --> whole sequnece has a length of 2*tau*order
pporder['order_start'] = 1
pporder['order_incr'] = 1
pporder['points'] = 40
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','pporder'])
pporder['tau'] = 530e-9
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','pporder'])
#NV14
pporder['order_incr'] = 2
pporder['points'] = 30
pporder['tau'] = 572e-9
do_multi_measurement(['poi_20170127_1402_38_232389'],['odmr','rabi','pporder'])
pporder['tau'] = 530e-9
do_multi_measurement(['poi_20170127_1402_38_232389'],['odmr','rabi','pporder'])

pptau['order'] = 30
#do_multi_measurement(['poi_20170127_1402_38_232389'],['odmr','rabi','pptau'])
pptau['order'] = 15
#do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','pptau'])

In [20]:
do_odmr()

PulsedODMR
seq_trigger


(False, 1019549809.7361903, -27.76650365371603, -450190.26380968094, 771792.6554457586)

In [ ]:
pulsedmaster.save_measurement_data('s','PulsePol-'+str(pptau['order'])+'_'+setup['NV_name']+'amp_'
                                       +str(pptau['mw_amp'])+'V',True)

In [ ]:
rabi['mw_amp']=0.05
rabi['length'] = 4000e-9
do_multi_measurement(['poi_20170127_1335_14_251083'],['rabi'])

In [ ]:
#NV21
#odmr['resonance_freq'] = 1017e6
#odmr['measurement_time'] = 60
#hhamp['sl_length'] = 15e-6
#hhamp['mw_freq'] = 1017e6
#hhamp['rabi_period'] = 291e-9
#hhamp['resonance_amp'] = 0.37005
#do_HHamp_refocus('poi_20170127_1335_14_251083')
#NV14
odmr['resonance_freq'] = 1022e6
odmr['measurement_time'] = 40
hhamp['sl_length'] = 20e-6
hhamp['mw_freq'] = 1022e6
hhamp['rabi_period'] = 252e-9
hhamp['resonance_amp'] = 0.33
do_HHamp_refocus('poi_20170127_1402_38_232389')

In [ ]:
hhtau['measurement_time'] = 7200
#NV21

odmr['resonance_freq'] =hhtau['mw_freq']=rabi['mw_freq']= 1012.5e6
odmr['measurement_time'] = 60
odmr['pi_length'] = 2445e-9/2.
hhtau['rabi_period'] = 282e-9
hhtau['sl_amp'] = 0.376514
hhtau['tau_start']=1e-6
hhtau['tau_res'] = 2e-6
hhtau['points'] = 30
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','hhtau'])
hhtau['sl_amp'] = 0.42
do_multi_measurement(['poi_20170127_1335_14_251083'],['odmr','rabi','hhtau'])


In [ ]:
#xy8['refocus_interval'] = 240
#xy8['order'] = 16
#xy8['tau_start'] = 170e-9
#xy8['tau_incr'] = 0.5e-9
#xy8['points'] = 30
#xy8['measurement_time'] = 10800
#do_multi_measurement(['poi_20170127_1402_38_232389'],['rabi','xy8'])
#xy8['order'] = 8
#xy8['tau_start'] = 168e-9
#xy8['tau_incr'] = 1e-9
#xy8['points'] = 22
#xy8['measurement_time'] = 3600
#do_multi_measurement(['poi_20170127_1335_14_251083'],['rabi','xy8'])
xy8['order'] = 16
xy8['tau_start'] = 170e-9
xy8['tau_incr'] = 0.5e-9
xy8['points'] = 30
xy8['measurement_time'] = 10800
do_multi_measurement(['poi_20170127_1346_49_682718'],['rabi','xy8'])


# Methods that still need to be reworked

In [ ]:
# for HHpol:
hhpol = OrderedDict()
hhpol['pulse_amp'] = 0.02
hhpol['sl_amp'] = 0.0395
hhpol['mw_freq'] = 1380.357e6
hhpol['rabi_period'] = 1244.0e-9
hhpol['sl_length'] = 22.0e-6
hhpol['num_pols'] = 100
hhpol['measurement_time'] = 10600
hhpol['refocus_interval'] = 600

hhpolup = OrderedDict()
hhpolup['pulse_amp'] = 0.05
hhpolup['sl_amp'] = 0.09057
hhpolup['mw_freq'] = 1103.897e6
hhpolup['rabi_period'] = 1198.0e-9
hhpolup['sl_length'] = 20.0e-6
hhpolup['num_pols'] = 100
hhpolup['measurement_time'] = 10600
hhpolup['refocus_interval'] = 600

hhpoldown = OrderedDict()
hhpoldown['pulse_amp'] = 0.02
hhpoldown['sl_amp'] = 0.0395
hhpoldown['mw_freq'] = 1380.357e6
hhpoldown['rabi_period'] = 1244.0e-9
hhpoldown['sl_length'] = 22.0e-6
hhpoldown['num_pols'] = 100
hhpoldown['measurement_time'] = 10600
hhpoldown['refocus_interval'] = 600

def do_HHpol(generate_new=True, save_tag=None):
    #Not working! --> Still need to be changed
    #
    #
    if save_tag is None:
        save_tag = ''
    # generate ensemble object
    reps_arr = np.arange(1, 2 * hhpol['num_pols'] + 1)
    if generate_new:
        sequencegenerator.delete_ensemble('HHpol')
        sequencegenerator.generate_HHpol('HHpol', hhpol['rabi_period'], hhpol['sl_length'], hhpol['sl_amp'], 
                                         hhpol['mw_freq'], hhpol['pulse_amp'], hhpol['num_pols'], 'a_ch1', 
                                         setup['laser_length'], 0.0, setup['aom_delay'], setup['wait_length'], 
                                         'd_ch1','d_ch3')
        # sample, upload and load waveform
        if 'HHpol' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of HHpol in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('HHpol', True)
    else:
        pulsedmaster.load_asset_into_channels('HHpol')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(reps_arr, 2 * hhpol['num_pols'], 1e-3, [], False, 700e-9)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + hhamp['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(save_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hhpol['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag == '':
        pulsedmaster.save_measurement_data('HHpol')
    else:
        pulsedmaster.save_measurement_data(save_tag)
    time.sleep(2)
    return user_terminated

def do_HHpolup(generate_new=True, save_tag=None):
    if save_tag is None:
        save_tag = ''
    # generate ensemble object
    reps_arr = np.arange(1, 2 * hhpolup['num_pols'] + 1)
    if generate_new:
        sequencegenerator.delete_ensemble('HHpolup')
        sequencegenerator.generate_HHpolup('HHpolup', hhpolup['rabi_period'], hhpolup['sl_length'], hhpolup['sl_amp'], 
                                           hhpolup['mw_freq'], hhpolup['pulse_amp'], hhpolup['num_pols'], 'a_ch1', 
                                           hhpolup['laser_length'], 0.0, setup['aom_delay'], hhpolup['wait_length'], 
                                           'd_ch1','d_ch3')
        # sample, upload and load waveform
        if 'HHpolup' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of HHpolup in notebook failed.')
            return
        pulsedmaster.sample_block_ensemble('HHpolup', True, False, True)
    else:
        pulsedmaster.load_asset_into_channels('HHpolup')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(reps_arr, hhpolup['num_pols'], 1e-3, [], False, 700e-9)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + hhpolup['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(save_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hhpolup['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    if save_tag == '':
        pulsedmaster.save_measurement_data('HHpolup')
    else:
        pulsedmaster.save_measurement_data(save_tag)
    time.sleep(2)
    return user_terminated

def do_HHpoldown(generate_new=True, save_tag=None):
    if save_tag is None:
        save_tag = ''
    # generate ensemble object
    reps_arr = np.arange(1, 2 * hhpoldown['num_pols'] + 1)
    if generate_new:
        sequencegenerator.delete_ensemble('HHpoldown')
        sequencegenerator.generate_HHpoldown('HHpoldown', hhpoldown['rabi_period'], hhpoldown['sl_length'], hhpoldown['sl_amp'], 
                                           hhpoldown['mw_freq'], hhpoldown['pulse_amp'], hhpoldown['num_pols'], 'a_ch1', 
                                           hhpoldown['laser_length'], 0.0, setup['aom_delay'], hhpoldown['wait_length'], 
                                           'd_ch1','d_ch3')
        # sample, upload and load waveform
        if 'HHpoldown' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of HHpoldown in notebook failed.')
            return
        pulsedmaster.sample_block_ensemble('HHpoldown', True, False, True)
    else:
        pulsedmaster.load_asset_into_channels('HHpoldown')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(reps_arr, hhpoldown['num_pols'], 1e-3, [], False, 700e-9)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['aom_delay'] + hhpoldown['laser_length'])
    pulsedmaster.analysis_interval_changed(20)
    # perform measurement
    pulsedmaster.start_measurement(save_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < hhpoldown['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    if save_tag == '':
        pulsedmaster.save_measurement_data('HHpoldown')
    else:
        pulsedmaster.save_measurement_data(save_tag)
    time.sleep(2)
    return user_terminated





def do_HHpol_refocus():
    refocus_position()
    end_measure, odmr_freq, odmr_contrast, odmr_shift, odmr_linewidth = do_odmr(True)
    if end_measure:
        return
    hhpol['mw_freq'] = odmr_freq
    hhpol_total_time = hhpol['measurement_time']
    hhpol_runtime = 0.0
    hhpol['measurement_time'] = hhpol['refocus_interval']
    end_measure = do_HHpol(True, 'HHpol_refocus')
    hhpol_runtime += hhpol['refocus_interval']
    while hhpol_total_time > hhpol_runtime:
        if end_measure:
            break
        refocus_position()
        end_measure, odmr_freq, odmr_contrast, odmr_shift, odmr_linewidth = do_odmr(False)
        if end_measure:
            break
        if odmr_contrast > 1 and odmr_shift < 20e6:
            hhpol['mw_freq'] = odmr_freq
            end_measure = do_HHpol(True, 'HHpol_refocus')
        else:
            end_measure = do_HHpol(False, 'HHpol_refocus')
        hhpol_runtime += hhpol['refocus_interval']
    hhpol['measurement_time'] = hhpol_total_time
    # save measurement
    pulsedmaster.save_measurement_data('HHpol_withrefocus_final')
    time.sleep(2)
    return
